In [ ]:
# Manipulation de données 
import pandas as pd 
import numpy as np



# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from PIL import Image



# Preprocessing des reviews 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

    # Pour filtrer la langue des avis
from langdetect import detect, DetectorFactory 
from langdetect.lang_detect_exception import LangDetectException



# Modelisation
    # Méthode classique 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering
import hdbscan
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.decomposition import PCA

    # Méthode avancée
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from transformers import BertTokenizer, BertModel, TFBertModel
import torch
from torch.utils.data import DataLoader, TensorDataset
from bertopic import BERTopic 




# Pour enregistrer et charger les modèles
import joblib

# CHARGEMENT DES DONNEES

In [ ]:
# Initialisation d'un dataframe vide
df = pd.read_csv('Data/avis_ugc_clean.csv')

In [ ]:
df.head()

# PREPROCESSING

Avant de passer nos données dans les modèles, nous devons les mettre en forme. Tout d’abord, j’ai constaté que la colonne `language` contient souvent des informations incorrectes sur la langue. Pour remédier à cela, nous allons utiliser la librairie `langdetect`, qui s’appuie sur la détection de langue de Google pour identifier la langue correcte.

In [ ]:
"""df_clean = df.copy()"""

# Importation du fichier déjà nettoyé car le code est long à tourner
df_clean = pd.read_csv('Data/avis_ugc_fr.csv')

In [ ]:
"""DetectorFactory.seed = 24

# Fonction pour détecter la langue d'un texte donné
def detect_language(text: str) -> str | None:
    try:
        # Tente de détecter la langue du texte
        return detect(text)
    except LangDetectException:
        # En cas d'erreur (par exemple, si le texte est vide ou ambigu), retourne None
        return None

# Applique la fonction de détection de langue à chaque avis dans la colonne 'review'
df_clean['language_detected'] = df_clean['review'].map(detect_language)

# Filtre les données pour ne conserver que les avis détectés en français ('fr')
df_clean = df[df['language_detected'] == 'fr']

# Exporte le DataFrame filtré dans un fichier CSV
df_clean.to_csv('Data/avis_ugc_fr.csv', index=False)"""

In [ ]:
print(f" Nombre d'avis non francais supprimés : {df.shape[0] - df_clean.shape[0]}")

In [ ]:
# Importation des stop words français de la librairie NLTK
stop_words = set(stopwords.words('french'))

# Ajout de stop words détectés dans les avis 
custom_stop_words = {",", ".", "a", "c'est", "!", "film", "cinéma", 
                     "si", "plus", "ugc", "ca", "là", "où", "deux", 
                     "ça", "h", "qu", "il", "''", '``', "-", "(", ")",
                     "..", "€", "?", "....", "//", ":", "/", ";", '’'
                    }
stop_words.update(custom_stop_words)

# Initialisation d'un lemmatiseur pour réduire les mots à leur forme de base
lemmatizer = WordNetLemmatizer()

def preprocess_reviews(reviews: pd.Series) -> pd.Series:
    """
    Cette fonction prend une série pandas contenant des avis textuels et applique un prétraitement
    pour chaque avis. Elle effectue les étapes suivantes :
    
    1. Convertit chaque avis en minuscules pour uniformiser le texte.
    2. Supprime les chiffres et remplace les apostrophes par des espaces pour éviter les séparations incorrectes.
    3. Tokenise chaque avis, c'est-à-dire qu'elle divise le texte en mots individuels.
    4. Supprime les stop words (mots courants sans importance) à partir d'une liste de stop words standard en français,
       enrichie de mots spécifiques au contexte.
    5. Applique la lemmatisation, qui réduit les mots à leur forme de base (par exemple, "films" devient "film").
    
    Paramètres :
    reviews (pd.Series) : Série pandas contenant les avis textuels.
    
    Retourne :
    pd.Series : Série pandas contenant des listes de mots prétraités pour chaque avis.
    """    
    
    # Convertir en minuscules et supprimer les chiffres et les apostrophes
    reviews = reviews.str.lower().str.replace(r'\d+', '', regex=True).str.replace("'", ' ')

    # Appliquer tokenisation, suppression des stop words, et lemmatisation
    reviews = reviews.apply(lambda review: [lemmatizer.lemmatize(word) 
                                            for word in word_tokenize(review) 
                                            if word not in stop_words
                                           ])


    return reviews

In [ ]:
reviews = preprocess_reviews(df_clean['review'])

In [ ]:
def my_word_cloud(reviews: pd.Series, title: str = None, subplot: tuple[int] = None, background_image_path: str = None) -> None:
    """
    Génère et affiche un nuage de mots basé sur le contenu textuel d'une série pandas contenant des avis.
    La fonction peut également afficher le nuage de mots sur une image de fond spécifiée.
    
    Paramètres :
    - reviews (pd.Series) : Série pandas contenant les avis textuels. Tous les avis sont concaténés pour créer le nuage de mots.
    - title (str, optionnel) : Titre à afficher au-dessus du nuage de mots. Si None, aucun titre n'est affiché.
    - subplot (tuple[int], optionnel) : Spécifie l'emplacement du nuage de mots dans une grille de sous-plots sous la forme (nrows, ncols, index).
      Utilisé lorsque le nuage de mots doit être affiché dans une figure avec plusieurs sous-graphiques.
    - background_image_path (str, optionnel) : Chemin d'accès à une image de fond pour le nuage de mots.
      Si fourni, le nuage de mots prendra la forme de cette image. Si None, le fond sera blanc par défaut.
    
    Retourne :
    - None : La fonction affiche directement le nuage de mots en utilisant `matplotlib.pyplot` et ne retourne rien.
    """
    
    # Concatène tous les avis en une seule chaîne de texte
    text = ' '.join(reviews.astype(str))
    
    # Charge l'image de fond si spécifiée, sinon crée un nuage de mots avec un fond blanc
    if background_image_path is not None:
        background_image = np.array(Image.open(background_image_path))
        wordcloud = WordCloud(width=800, height=400, background_color='white', mask=background_image, contour_width=1, contour_color='black').generate(text)
    else:
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    

    plt.figure(figsize=(10, 5))
    
    # Positionne le nuage de mots dans un sous-plot si `subplot` est spécifié
    if subplot is not None :
        plt.subplot(subplot[0], subplot[1], subplot[2])
    
    # Affiche le nuage de mots et enlève les axes
    plt.imshow(wordcloud)
    plt.axis('off')
    
    # Ajoute un titre si spécifié
    if title is not None:
        plt.title(title)
    
    plt.show()

In [ ]:
my_word_cloud(reviews)

In [ ]:
# Boucle pour afficher un word cloud pour chaque note
for i in np.sort(df_clean['rating'].unique()):
    reviews_per_note =  preprocess_reviews(df_clean[df_clean['rating'] == i]['review'])
    
    title = f' Word Cloud des avis {i} étoiles'
    my_word_cloud(reviews_per_note, title, subplot=(3,2,i))

* Le mot "salle(s)" apparaît aussi bien dans les avis positifs que négatifs, ce qui suggère qu'il n'est pas un indicateur fiable de sentiment. Il serait donc pertinent de le retirer, au moins pour la création du nuage de mots, afin d'obtenir une visualisation plus représentative des termes associés aux avis positifs ou négatifs.

In [ ]:
# Boucle pour afficher un word cloud pour chaque note
for i in np.sort(df_clean['rating'].unique()):
    reviews_per_note =  preprocess_reviews(df_clean[df_clean['rating'] == i]['review'])
    
    reviews_per_note = reviews_per_note.apply(lambda words: [word for word in words if word not in ['salle', 'salles']])
    
    title = f' Word Cloud des avis {i} étoiles'
    my_word_cloud(reviews_per_note, title, subplot=(3,2,i))

1. **Word Cloud des avis 1 étoile :**

    * Les mots les plus fréquents incluent "place", "séance", et "bien". Cela pourrait suggérer des commentaires sur l'emplacement, l'organisation des séances, ou même des aspects de confort. Cependant, dans un contexte de note basse, ces termes sont probablement utilisés de manière négative (ex. : mauvaise organisation des séances, inconfort des places).
    * Les termes "dommage", "problème", et "mal" apparaissent également, ce qui renforce l'idée de critiques.

2. **Word Cloud des avis 2 étoiles :**

    * On retrouve des mots similaires comme "place" et "tout". "Dommage" est également présent, ce qui indique des critiques mitigées.
    * Le mot "bien" reste fréquent, mais dans ce contexte, il pourrait être utilisé pour souligner des aspects corrects dans un avis globalement négatif.

3. **Word Cloud des avis 3 étoiles :**

    * Les termes "bien" et "peu" sont dominants, suggérant des avis modérés où des aspects positifs sont soulignés, mais avec des réserves (ex. : "peu d'espace", "bien mais pourrait être mieux").
    * D’autres mots comme "dommage", "prix", et "petite" indiquent probablement des points d'amélioration signalés par les utilisateurs.

4. **Word Cloud des avis 4 étoiles :**

    * Le mot "bien" devient encore plus dominant, accompagné de "très", "bon", et "propre". Cela suggère des avis majoritairement positifs, où la qualité et le confort sont reconnus.
    * Des termes comme "agréable" et "choix" apparaissent, ce qui peut indiquer une satisfaction avec les options disponibles et l’atmosphère générale.

5. **Word Cloud des avis 5 étoiles :**

    * Les mots les plus fréquents sont "très", "super", "bien", et "agréable", soulignant une satisfaction élevée.
    * On voit aussi "parfait", "personnel", et "choix", ce qui indique une expérience globalement positive avec de nombreux aspects appréciés, y compris le service.

# MODELISATION : Identifier les différentes thématiques des avis

## Utilisation de méthode classique (TF-IDF)

In [ ]:
# transformation des avis en une liste de texte
reviews = [' '.join(review) for review in reviews]

# On applique la transformation TF-IDF aux avis pour obtenir une matrice sparse des scores TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X = tfidf_vectorizer.fit_transform(reviews)

### HDBSCAN

> DBSCAN identifie des clusters en regroupant les points qui se trouvent dans des zones de densité uniforme, ce qui limite sa capacité pour des données à densité variable.
>
> HDBSCAN est une version améliorée qui détecte les clusters de densité variable de manière hiérarchique, permettant une identification plus flexible et automatique des groupes dans des ensembles de données complexes. HDBSCAN n’a pas besoin de définir `eps` et se base principalement sur deux paramètres : `min_cluster_size` (taille minimale pour un cluster) et `min_samples` (contrôle de la sensibilité au bruit).

In [ ]:
pca_hdbscan = PCA(n_components=100) 
reduced_vectors = pca_hdbscan.fit_transform(X)

sum(pca_hdbscan.explained_variance_ratio_)

In [ ]:
"""hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=100, min_samples=3)
hdbscan_model.fit(reduced_vectors)"""

hdbscan_model = joblib.load('Entrainement_modeles_tfidf/hdbscan_tfidf.joblib')

# Résultats des clusters
labels_hdbscan = hdbscan_model.labels_

calinski_score = calinski_harabasz_score(reduced_vectors, labels_hdbscan)
print("Score de Calinski-Harabasz :", calinski_score) 

"""joblib.dump(hdbscan_model, 'Entrainement_modeles_tfidf/hdbscan_tfidf.joblib')"""

In [ ]:
df_clean['HDBSCAN_labels'] = labels_hdbscan

df_clean.groupby('HDBSCAN_labels').agg(rating_mean=('rating', 'mean'),
                                rating_Count=('rating', 'count')
                               ).sort_values(by='rating_mean', ascending=False)

### K-mean

In [ ]:
# Calcul du score de silhouette pour déterminer le nombre optimal de clusters
"""silhouette_scores = []
range_n_clusters = range(5, 20)

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=24)
    cluster_labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print('nombre clusters :', n_clusters, silhouette_avg)

# Tracer le score de silhouette pour chaque nombre de clusters
plt.plot(range_n_clusters, silhouette_scores, marker='o')
plt.title("Silhouette Method")
plt.xlabel("Nombre de clusters")
plt.ylabel("Silhouette Score")
plt.show()"""

# Choisir le nombre optimal de clusters
"""optimal_clusters = range_n_clusters[np.argmax(silhouette_scores)]"""

optimal_clusters = joblib.load('Entrainement_modeles_tfidf/optimal_clusters_kmean_tfidf.joblib')

"""joblib.dump(optimal_clusters, 'Entrainement_modeles_tfidf/optimal_clusters_kmean_tfidf.joblib')"""

print(f"Nombre de clusters optimal basé sur le score de silhouette: {optimal_clusters}")

In [ ]:
kmeans = KMeans(n_clusters=optimal_clusters, random_state=24)

df_clean['Kmean_labels'] = kmeans.fit_predict(X)

df_clean.groupby('Kmean_labels').agg(rating_mean=('rating', 'mean'),
                                rating_Count=('rating', 'count')
                               ).sort_values(by='rating_mean', ascending=False)

### CHA

* Je n'ai pas pu implémenter cet algorithme en raison de problèmes de mémoire que je n'ai pas réussi à résoudr.CHA attend un vecteur dense ce qui consomme beaucoup trop de mémoire. Je partage néanmoins le code ci-dessous pour référence.

```python
# Clustering hiérarchique avec AgglomerativeClustering
ca_clustering = AgglomerativeClustering(n_clusters=10)
labels_cah = ca_clustering.fit_predict(X.toarray())

# Calcul du score de silhouette pour évaluer la qualité des clusters
silhouette_avg = silhouette_score(X.toarray(), labels_cah)
print(f"Score de silhouette pour le CAH : {silhouette_avg}")

# Visualisation du dendrogramme pour voir la hiérarchie des clusters
linked = linkage(X.toarray(), 'ward')

plt.figure(figsize=(10, 7))
dendrogram(linked, orientation='top', labels=reviews, distance_sort='descending', show_leaf_counts=True)
plt.title("Dendrogramme du clustering hiérarchique")
plt.xlabel("Index des reviews")
plt.ylabel("Distance")
plt.show()
```

### LDA

In [ ]:
"""lda = LatentDirichletAllocation(n_components=10, random_state=24)
lda.fit(X)

topic_values = lda.transform(X)"""

lda, topic_values = joblib.load('Entrainement_modeles_tfidf/lda_tfidf.joblib')

"""joblib.dump((lda, topic_values), 'Entrainement_modeles_tfidf/lda_tfidf.joblib')"""

In [ ]:
df_clean['LDA_labels'] = np.argmax(topic_values, axis=1)

df_clean.groupby('LDA_labels').agg(rating_mean=('rating', 'mean'),
                                rating_Count=('rating', 'count')
                               ).sort_values(by='rating_mean', ascending=False)

### Conclusion

Nous obtenons de meilleurs résultats avec le modèle Latent Dirichlet Allocation (LDA), qui est bien adapté à notre problème de regroupement thématique des avis. En effet, LDA est conçu pour identifier des thèmes sous-jacents dans des documents textuels, ce qui permet de regrouper les avis en thématiques relativement homogènes. LDA a également permis de mettre en évidence des différences entre les groupes en termes de notes associées. Cependant, pour exploiter pleinement le potentiel de LDA, il reste essentiel d’optimiser ses hyperparamètres afin d’obtenir des clusters plus précis.

HDBSCAN, bien que potentiellement adapté pour identifier des clusters de forme et de densité variées, n’a pas encore donné de résultats satisfaisants probablement en raison de réglages d’hyperparamètres sous-optimaux. HDBSCAN peut être utile pour notre problème si nous parvenons à optimiser ces paramètres, car il pourrait détecter des groupes d'avis bien distincts, notamment des groupes d’avis extrêmes (très positifs ou très négatifs) sans forcer une structure sphérique comme K-Means. Il reste néanmoins limité dans un contexte de données textuelles de grande dimension.

K-Means a également été utilisé, mais cet algorithme n’est pas parfaitement adapté à notre problème. En effet, K-Means fonctionne mieux avec des clusters de forme sphérique et nécessite un calcul intensif des distances, ce qui devient coûteux en termes de mémoire et de temps d'exécution lorsque le volume de données textuelles et la dimensionnalité des vecteurs sont élevés (nous utilisons des vecteurs de dimension 1000). K-Means est moins performant pour capturer des relations complexes entre les thèmes des avis et se révèle inadapté aux données textuelles non linéaires.

Un autre défi majeur est lié à l’utilisation de TF-IDF, qui présente des limites pour représenter le texte. Bien qu’il permette de transformer les avis en vecteurs, TF-IDF ne capture pas les relations contextuelles entre les mots et se limite à la fréquence de chaque mot dans les documents. Par conséquent, des mots ayant des significations similaires mais des formes différentes (par exemple, "excellent" et "super") ne sont pas traités comme liés, ce qui réduit la cohérence des clusters. De plus, TF-IDF produit des vecteurs de grande dimension, entraînant une consommation importante de mémoire sans pour autant améliorer la qualité des clusters.

Pour surmonter ces limitations, nous allons explorer des méthodes plus avancées, notamment des techniques de Deep Learning telles que les embeddings de texte. Les embeddings permettent de réduire la dimensionnalité tout en capturant des relations sémantiques et contextuelles riches entre les mots. En utilisant des embeddings préentraînés ou des modèles de transformers, nous pourrons mieux représenter les avis sous forme de vecteurs de faible dimension mais significatifs, optimisant ainsi les ressources mémoire et accélérant les calculs. Cette approche devrait nous permettre d’obtenir des clusters plus cohérents et représentatifs des thèmes exprimés dans les avis.

## Utilisation de méthode plus avancée (deep learning)

### Word2Vec

#### Entrainement d'un modèle Word2Vec

In [ ]:
# On transforme chaque review en liste de mots
sentences = [review.split() for review in reviews]

# Initialisation d'un modèle Word2Vec
w2v_model = Word2Vec(vector_size=200, seed=24)

# Construction du vocabulaire à partir des phrases.
w2v_model.build_vocab(sentences)

# Récupération du vocabulaire
words = list(w2v_model.wv.index_to_key)
vocab_size = len(words)
print(f"Taille du vocabulaire : {vocab_size}")

# Entrainement du modèle
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30)

In [ ]:
def get_review_vector(review: list[str], word_vectors: dict[str, np.ndarray], model: Word2Vec) -> np.ndarray:
    
    """
    Cette fonction génère un vecteur pour chaque review en calculant la moyenne des vecteurs de mots
    présents dans la review.

    Arguments :
    review -- Liste de mots de la revue.
    word_vectors -- Dictionnaire contenant les représentations vectorielles des mots (généré par le modèle Word2Vec).
    model -- Le modèle Word2Vec utilisé pour extraire la taille des vecteurs (utile si la revue n'a pas de mots valides).
    
    Retourne :
    Un vecteur représentant la revue. Si aucun mot de la revue n'est présent dans le vocabulaire du modèle,
    retourne un vecteur nul (composé de zéros).
    """
    
    vectors = [word_vectors[word] for word in review if word in word_vectors]
    
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    
    return np.mean(vectors, axis=0)

word_vectors = w2v_model.wv

# On transforme chaque review en un vecteur
review_vectors = np.array([get_review_vector(review, word_vectors, w2v_model) for review in sentences])

* Pour effectuer le clustering des reviews après word embedding, j'ai choisi de prendre la moyenne des vecteurs de chaque mot pour obtenir un vecteur représentatif par review. Cette approche simplifie le processus de clustering en réduisant chaque review à un vecteur de dimension fixe, facilitant ainsi l’application d’algorithmes comme HDBSCAN. Bien qu'elle implique une perte d’information contextuelle en résumant chaque review en un seul vecteur, cette méthode est bien adaptée pour extraire les thèmes principaux de manière efficace et optimise les performances en termes de mémoire et de temps de calcul.

##### HDBSCAN

In [ ]:
"""hdbscan_w2v = hdbscan.HDBSCAN(min_samples=3, min_cluster_size=100)
hdbscan_w2v.fit(review_vectors)"""

hdbscan_w2v = joblib.load('Entrainement_modeles_deep/hdbscan_w2v.joblib')

# Résultats des clusters
labels_hdbscan_w2v = hdbscan_w2v.labels_

calinski_score = calinski_harabasz_score(review_vectors, labels_hdbscan_w2v)
print("Score de Calinski-Harabasz :", calinski_score) 

"""joblib.dump(hdbscan_w2v, 'Entrainement_modeles_deep/hdbscan_w2v.joblib')"""

In [ ]:
df_clean['DBSCAN_w2v_labels'] = labels_hdbscan_w2v

df_clean.groupby('DBSCAN_w2v_labels').agg(rating_mean=('rating', 'mean'),
                                          rating_Count=('rating', 'count')
                                         ).sort_values(by='rating_mean', ascending=False)

##### LDA

In [ ]:
"""# Normalisation pour garantir que toutes les valeurs sont non négatives
scaler = MinMaxScaler()
review_vectors_scaled = scaler.fit_transform(review_vectors)

lda_w2v = LatentDirichletAllocation(n_components=10, random_state=24)
lda_w2v.fit(review_vectors_scaled)

topic_values_w2v = lda_w2v.transform(review_vectors_scaled)"""

lda_w2v, topic_values_w2v = joblib.load('Entrainement_modeles_deep/lda_w2v.joblib')

"""joblib.dump((lda_w2v, topic_values_w2v), 'Entrainement_modeles_deep/lda_w2v.joblib')"""

In [ ]:
df_clean['LDA_w2v_labels'] = np.argmax(topic_values_w2v, axis=1)

df_clean.groupby('LDA_w2v_labels').agg(rating_mean=('rating', 'mean'),
                                       rating_Count=('rating', 'count')
                                      ).sort_values(by='rating_mean', ascending=False)

#### Utilisation d'un modèle pré entrainer

In [ ]:
w2v_model_pt = api.load("glove-wiki-gigaword-200")

In [ ]:
# Transformation des review en un vecteur
review_vectors_pt = np.array([get_review_vector(review, w2v_model_pt, w2v_model_pt) for review in sentences])

##### HDBSCAN

In [ ]:
"""hdbscan_w2v_pt = hdbscan.HDBSCAN(min_samples=3, min_cluster_size=100)
hdbscan_w2v_pt.fit(review_vectors_pt)"""

hdbscan_w2v_pt = joblib.load('Entrainement_modeles_deep/hdbscan_w2v_pt.joblib')

# Résultats des clusters
labels_hdbscan_w2v_pt = hdbscan_w2v_pt.labels_

calinski_score = calinski_harabasz_score(review_vectors_pt, labels_hdbscan_w2v_pt)
print("Score de Calinski-Harabasz :", calinski_score) 

"""joblib.dump(hdbscan_w2v_pt, 'Entrainement_modeles_deep/hdbscan_w2v_pt.joblib')"""

In [ ]:
df_clean['DBSCAN_w2v_pt_labels'] = labels_hdbscan_w2v_pt

df_clean.groupby('DBSCAN_w2v_pt_labels').agg(rating_mean=('rating', 'mean'),
                                rating_Count=('rating', 'count')
                               ).sort_values(by='rating_mean', ascending=False)

##### LDA

In [ ]:
"""scaler = MinMaxScaler()
review_vectors_scaled_pt = scaler.fit_transform(review_vectors_pt)

lda_w2v_pt = LatentDirichletAllocation(n_components=10, random_state=24)
lda_w2v_pt.fit(review_vectors_scaled_pt)

topic_values_w2v_pt = lda_w2v_pt.transform(review_vectors_scaled_pt)"""

lda_w2v_pt, topic_values_w2v_pt = joblib.load('Entrainement_modeles_deep/lda_w2v_pt.joblib')

"""joblib.dump((lda_w2v_pt, topic_values_w2v_pt), 'Entrainement_modeles_deep/lda_w2v_pt.joblib')"""

In [ ]:
df_clean['LDA_w2v_pt_labels'] = np.argmax(topic_values_w2v_pt, axis=1)

df_clean.groupby('LDA_w2v_pt_labels').agg(rating_mean=('rating', 'mean'),
                                          rating_Count=('rating', 'count')
                                         ).sort_values(by='rating_mean', ascending=False)

### BERT

In [ ]:
# Si un GPU est disponible, on utilise CUDA pour le traitement, sinon, on revient sur le CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Chargement du tokenizer et du modèle BERT pré-entraîné
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Envoyer le modèle sur le GPU (ou CPU si le GPU n'est pas disponible)
bert_model = bert_model.to(device)

def process_batch(batch_reviews):
    """
    Cette fonction prend un lot de reviews sous forme de liste de chaînes de caractères,
    les tokenise, passe les tokens dans le modèle BERT, et renvoie les embeddings moyens pour chaque review.
    
    Paramètres :
    - batch_reviews (list): Liste de chaînes de caractères représentant les avis (reviews).
    
    Retour :
    - sentence_embeddings (Tensor): Tensor de taille (batch_size, embedding_dim) contenant les embeddings moyens
      de chaque review dans le lot.
    """
    
    # Utilisation du tokenizer BERT pour convertir les textes en séquences de tokens    
    inputs = bert_tokenizer(batch_reviews, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Envoyer les entrées sur le même périphérique (GPU ou CPU)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Traitement du lot avec le modèle BERT
    # On utilise 'torch.no_grad()' pour désactiver le calcul des gradients, ce qui économise de la mémoire pendant l'inférence
    with torch.no_grad():
        outputs = bert_model(**inputs)

    # Récupération des embeddings
    embeddings = outputs.last_hidden_state

    # Moyenne des embeddings pour obtenir un vecteur par review
    sentence_embeddings = embeddings.mean(dim=1)
    
    return sentence_embeddings

In [ ]:
"""# Diviser les avis en lots de taille 1000 pour éviter un problème de mémoire
batch_size = 500
all_embeddings = []

# Diviser les données en lots de taille 'batch_size'
for i in range(0, len(reviews), batch_size):
    # Extraire le lot actuel d'avis
    batch_reviews = reviews[i:i + batch_size]
    
    # Traiter chaque lot et récupérer les embeddings
    batch_embeddings = process_batch(batch_reviews)
    
    # Ajouter les embeddings du batch au résultat final
    all_embeddings.append(batch_embeddings)

    # Libérer la mémoire GPU après chaque lot
    torch.cuda.empty_cache()

# Concaténer les embeddings de toutes les phrases
sentence_embeddings = torch.cat(all_embeddings, dim=0)"""

sentence_embeddings = joblib.load('Entrainement_modeles_deep/embeddings_bert.joblib')

# Afficher la forme des embeddings de phrases
print("Shape des embeddings pour chaque review :", sentence_embeddings.shape)

"""joblib.dump(sentence_embeddings, 'Entrainement_modeles_deep/embeddings_bert.joblib')"""

#### HDBSCAN

In [ ]:
"""hdbscan_bert = hdbscan.HDBSCAN(min_samples=3, min_cluster_size=100)
hdbscan_bert.fit(sentence_embeddings)"""

hdbscan_bert = joblib.load('Entrainement_modeles_deep/hdbscan_bert.joblib')

# Résultats des clusters
labels_hdbscan_bert = hdbscan_bert.labels_

calinski_score = calinski_harabasz_score(sentence_embeddings, labels_hdbscan_bert)
print("Score de Calinski-Harabasz :", calinski_score) 

"""joblib.dump(hdbscan_bert, 'Entrainement_modeles_deep/hdbscan_bert.joblib')"""

In [ ]:
df_clean['HDBSCAN_BERT_labels'] = labels_hdbscan_bert

df_clean.groupby('HDBSCAN_BERT_labels').agg(rating_mean=('rating', 'mean'),
                                          rating_Count=('rating', 'count')
                                         ).sort_values(by='rating_mean', ascending=False)

#### LDA

In [ ]:
"""scaler = MinMaxScaler()
sentence_embedding_scaled = scaler.fit_transform(sentence_embeddings)
 
lda_bert = LatentDirichletAllocation(n_components=10, random_state=24)
lda_bert.fit(sentence_embedding_scaled)

topic_values_bert = lda_bert.transform(sentence_embedding_scaled)"""

lda_bert, topic_values_bert = joblib.load('Entrainement_modeles_deep/lda_bert.joblib')

"""joblib.dump((lda_bert, topic_values_bert), 'Entrainement_modeles_deep/lda_bert.joblib')"""

In [ ]:
df_clean['LDA_BERT_labels'] = np.argmax(topic_values_bert, axis=1)

df_clean.groupby('LDA_BERT_labels').agg(rating_mean=('rating', 'mean'),
                                          rating_Count=('rating', 'count')
                                         ).sort_values(by='rating_mean', ascending=False)

#### BERTopic

In [ ]:
topic_model = BERTopic(language='french',  nr_topics=10)
topics, proba = topic_model.fit_transform(reviews)

In [ ]:
df_clean['BERTopic_labels'] = topics

df_clean.groupby('BERTopic_labels').agg(
    rating_mean=('rating', 'mean'),
    rating_Count=('rating', 'count')
).sort_values(by='rating_mean', ascending=False)

# CONCLUSION